<a href="https://colab.research.google.com/github/Erfan7135/Langchain/blob/main/langchain.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Basic Setup

In [1]:
!python --version

Python 3.11.13


installing necessary packages

In [2]:
!pip install -q --upgrade langchain langchain-core langchain-google-genai google-generativeai==0.8.5

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.0/42.0 kB 1.0 MB/s eta 0:00:00


Verify Installation

In [3]:
!pip show google-generativeai google-ai-generativelanguage langchain-google-genai

Name: google-generativeai
Version: 0.8.5
Summary: Google Generative AI High level API client library and tools.
Home-page: https://github.com/google/generative-ai-python
Author: Google LLC
Author-email: googleapis-packages@google.com
License: Apache 2.0
Location: /usr/local/lib/python3.11/dist-packages
Requires: google-ai-generativelanguage, google-api-core, google-api-python-client, google-auth, protobuf, pydantic, tqdm, typing-extensions
Required-by: langchain-google-genai
---
Name: google-ai-generativelanguage
Version: 0.6.15
Summary: Google Ai Generativelanguage API client library
Home-page: https://github.com/googleapis/google-cloud-python/tree/main/packages/google-ai-generativelanguage
Author: Google LLC
Author-email: googleapis-packages@google.com
License: Apache 2.0
Location: /usr/local/lib/python3.11/dist-packages
Requires: google-api-core, google-auth, proto-plus, protobuf
Required-by: google-generativeai
---
Name: langchain-google-genai
Version: 2.0.10
Summary: An integratio

Import Neccessary Files

In [4]:
import langchain
import langchain_google_genai
print("LangChain and Gemini integration installed successfully!")

LangChain and Gemini integration installed successfully!


Import gemini key

In [19]:
from google.colab import userdata
api_key = userdata.get('gemini_api')
# print(api_key)

Initialized with langchain

In [46]:
from langchain_google_genai import ChatGoogleGenerativeAI

#Initialize the model
llm = ChatGoogleGenerativeAI(
    model = "gemini-2.5-flash",
    google_api_key = api_key,
    temparature = 0.7 # 0=deterministic, 1= random
)

Send Queries to verify setup

In [21]:
response = llm.invoke("Hello, What is langchain?")
print(response.content)

LangChain is an open-source **framework (or library) designed to simplify the development of applications powered by Large Language Models (LLMs)**.

Think of it as a toolkit that helps you take powerful LLMs (like GPT-4, Claude, Llama, etc.) and connect them with other data sources, computational tools, and your own logic to build more complex and useful applications.

Here's a breakdown of what that means and why it's so useful:

### Why LangChain? (The Problem It Solves)

While LLMs are incredibly powerful, they have some inherent limitations when used in isolation:

1.  **Lack of Real-time Information:** Their knowledge is typically cut off at their last training date. They can't browse the web or access current events.
2.  **No Memory:** By default, LLMs are stateless. Each interaction is separate, so they don't remember past conversations.
3.  **Inability to Perform Actions:** They can't directly execute code, call APIs, or interact with external systems (like sending an email or

In [38]:
from langchain.chains import LLMChain, SequentialChain
from langchain_core.output_parsers import StrOutputParser

# Chains... (prompt | llm)

Import Necessaries

In [8]:
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain

In [9]:
code_prompt = PromptTemplate(
    template="Write a very short {language} function that will {task}",
    input_variables=["language", "task"]
)

In [39]:
code_chain = code_prompt | llm | StrOutputParser()

In [11]:
result = code_chain.invoke({
    "language": "python",
    "task": "return a list of numbers"
})
print(result)
print(result.content)

content='```python\ndef get_numbers():\n  return [1, 2, 3, 4, 5] \n```' additional_kwargs={} response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'safety_ratings': []} id='run--d5e71c90-7dad-4062-9c23-3c3002a093df-0' usage_metadata={'input_tokens': 14, 'output_tokens': 29, 'total_tokens': 138, 'input_token_details': {'cache_read': 0}}
```python
def get_numbers():
  return [1, 2, 3, 4, 5] 
```


# Connecting Chain Together...

In [51]:
comment_prompt = PromptTemplate(
    input_variables=["code"],
    template="Explain the following code:\n{code}"
)
test_prompt = PromptTemplate(
    input_variables=["language", "code"],
    template="Write a test for following {language} code:\n{code}"
)

In [52]:
comment_chain = comment_prompt | llm | StrOutputParser()
test_chain = test_prompt | llm | StrOutputParser()

In [63]:
language = "python"
task = "return a list of numbers"

In [57]:
combined_chain_1 = code_chain | comment_chain

In [54]:
result = combined_chain_1.invoke({
    "language": language,
    "task": task
})
print(result)
#

This Python code defines a simple function named `get_numbers`.

Let's break down each part:

1.  **`def get_numbers():`**
    *   `def`: This keyword is used to *define* a function in Python.
    *   `get_numbers`: This is the *name* of the function. It's a good practice to choose descriptive names that indicate what the function does.
    *   `()`: These parentheses indicate that the function takes *no arguments* (or parameters). If it needed inputs, they would be listed inside these parentheses (e.g., `def add_numbers(a, b):`).
    *   `:`: This colon marks the end of the function header and the beginning of its body (the code that belongs to the function).

2.  **`"""Returns a list of numbers."""`**
    *   This is a **docstring** (documentation string). It's a multi-line string (enclosed in triple quotes `"""..."""`) that immediately follows the function definition.
    *   **Purpose:** Docstrings are used to provide a concise summary of what the function does, its arguments, and 

for multiple input in second chain, first chain should be defined in a function

In [71]:
def mod_output(input):
  language = input["language"]
  task = input["task"]
  code = code_chain.invoke(input)
  return {"language" : language, "code" : code}

In [72]:
from langchain_core.runnables import RunnableLambda

combined_chain_2 = RunnableLambda(mod_output) | test_chain
result = combined_chain_2.invoke({"language": language, "task": task})
print(result)
#

Okay, here are tests for the `get_numbers` function using both the built-in `unittest` framework and the popular `pytest` framework.

First, let's assume your `get_numbers` function is in a file named `my_module.py`:

**`my_module.py`:**
```python
def get_numbers():
    """
    Returns a predefined list of numbers.
    """
    return [1, 2, 3, 4, 5]
```

---

## 1. Using `unittest` (Built-in Python Testing Framework)

Create a new file named `test_my_module_unittest.py` (or similar) in the same directory:

**`test_my_module_unittest.py`:**
```python
import unittest
from my_module import get_numbers  # Import the function from your module

class TestGetNumbers(unittest.TestCase):
    """
    Test suite for the get_numbers function.
    """

    def test_returns_correct_list(self):
        """
        Test that get_numbers returns the exact expected list.
        """
        expected_list = [1, 2, 3, 4, 5]
        actual_list = get_numbers()
        self.assertEqual(actual_list, expected